<a href="https://colab.research.google.com/github/OSegun/Data-Engineering-Projects/blob/main/Covid_db.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# importing necessary libaries
import boto3
import time
import pandas as pd
from io import StringIO

In [ ]:
# AWS parameters to be used
AWS_ACCESS_KEY =
AWS_SECRET_KEY = 
AWS_REGION = 
SCHEMA_NAME = 
S3_STAGING_DIR = 
S3_BUCKET_NAME = 
S3_OUTPUT_DIRCTORY = 

In [ ]:
athena_client = boto3.client(
    "athena", 
    aws_access_key_id = AWS_ACCESS_KEY,
    aws_secret_access_key = AWS_SECRET_KEY,
    region_name = AWS_REGION,
)

In [ ]:
Dict = {}

def load_query_result(
    client: boto3.client, query_response: Dict
) -> pd.DataFrame:
    
    status = "RUNNING"
    while True:
        try:
            client.get_query_execution(QueryExecutionId=query_response["QueryExecutionId"])
            status = client.get_query_execution(QueryExecutionId=query_response["QueryExecutionId"])['QueryExecution']['Status']['State']
            #client.get_query_execution(QueryExecutionId = query_response)
            if status == "SUCCEEDED":
                client.get_query_results(
                QueryExecutionId = query_response["QueryExecutionId"]
                )
            else:
                break
        except Exception as err:
            if "not yet finished" in str(err):
                time.sleep(0.005)
            else:
                raise err
    temp_file_location: str = "athena_query_results.csv"
    s3_client = boto3.client(
       "s3",
       aws_access_key_id = AWS_ACCESS_KEY,
       aws_secret_access_key = AWS_SECRET_KEY,
       region_name = AWS_REGION, 
    )
    s3_client.download_file(
        S3_BUCKET_NAME,
        f"{S3_OUTPUT_DIRCTORY}/{query_response['QueryExecutionId']}.csv", 
        temp_file_location,
    )
    return pd.read_csv(temp_file_location)

## ETL & Data Analysis

In [ ]:
response = athena_client.start_query_execution(
    QueryString = "SELECT * FROM enigma_jhud",
    QueryExecutionContext = {"Database": SCHEMA_NAME},
    ResultConfiguration = {
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    }
)

df_data = load_query_result(athena_client, response)

In [ ]:
response

{'QueryExecutionId': '9f4316ad-dbfc-4614-b8e9-6aadc15c691b',
 'ResponseMetadata': {'RequestId': 'a2f3da2b-87ee-43bc-b146-38c992f30393',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 21 Mar 2023 16:34:15 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '59',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'a2f3da2b-87ee-43bc-b146-38c992f30393'},
  'RetryAttempts': 0}}

In [ ]:
enigma_jhud = load_query_result(athena_client, response)

In [ ]:
enigma_jhud.head()

,fips,admin2,province_state,country_region,last_update,latitude,longitude,confirmed,deaths,recovered,active,combined_key,partition_0
0,NaN,NaN,Anhui,China,2020-01-22T17:00:00,31.826,117.226,1.0,NaN,NaN,NaN,"""Anhui",csv
1,NaN,NaN,Beijing,China,2020-01-22T17:00:00,40.182,116.414,14.0,NaN,NaN,NaN,"""Beijing",csv
2,NaN,NaN,Chongqing,China,2020-01-22T17:00:00,30.057,107.874,6.0,NaN,NaN,NaN,"""Chongqing",csv
3,NaN,NaN,Fujian,China,2020-01-22T17:00:00,26.079,117.987,1.0,NaN,NaN,NaN,"""Fujian",csv
4,NaN,NaN,Gansu,China,2020-01-22T17:00:00,36.061,103.834,NaN,NaN,NaN,NaN,"""Gansu",csv


In [ ]:
response = athena_client.start_query_execution(
    QueryString = "SELECT * FROM nytimes_data_in_usa_countryus_county",
    QueryExecutionContext = {"Database": SCHEMA_NAME},
    ResultConfiguration = {
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    }
)

nytimes_data_in_usa_countryus_county = load_query_result(athena_client, response)

In [ ]:
response = athena_client.start_query_execution(
    QueryString = "SELECT * FROM nytimes_data_in_usa_statesus_states",
    QueryExecutionContext = {"Database": SCHEMA_NAME},
    ResultConfiguration = {
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    }
)

nytimes_data_in_usa_statesus_states = load_query_result(athena_client, response)

In [ ]:
response = athena_client.start_query_execution(
    QueryString = 'SELECT * FROM state_abv',
    QueryExecutionContext = {"Database": SCHEMA_NAME},
    ResultConfiguration = {
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    }
)

covid_db_state_abv = load_query_result(athena_client, response)

In [ ]:
covid_db_state_abv.head()

,col0,col1
0,State,Abbreviation
1,Alabama,AL
2,Alaska,AK
3,Arizona,AZ
4,Arkansas,AR


In [ ]:
header = covid_db_state_abv.iloc[0]
covid_db_state_abv = covid_db_state_abv[1:]
covid_db_state_abv.columns = header

In [ ]:
covid_db_state_abv.head()

,State,Abbreviation
1,Alabama,AL
2,Alaska,AK
3,Arizona,AZ
4,Arkansas,AR
5,California,CA


In [ ]:
response = athena_client.start_query_execution(
    QueryString = 'SELECT * FROM countrycode',
    QueryExecutionContext = {"Database": SCHEMA_NAME},
    ResultConfiguration = {
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    }
)

covid_db_countrycode = load_query_result(athena_client, response)

In [ ]:
response = athena_client.start_query_execution(
    QueryString = 'SELECT * FROM countypopulation',
    QueryExecutionContext = {"Database": SCHEMA_NAME},
    ResultConfiguration = {
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    }
)

covid_db_countypopulation = load_query_result(athena_client, response)

In [ ]:
response = athena_client.start_query_execution(
    QueryString = "SELECT * FROM testing_data_states_dailystates_daily",
    QueryExecutionContext = {"Database": SCHEMA_NAME},
    ResultConfiguration = {
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    }
)

covid_db_testing_data_states_dailystates_daily = load_query_result(athena_client, response)

In [ ]:
response = athena_client.start_query_execution(
    QueryString = 'SELECT * FROM testing_data_us_total_latestus_total_latest',
    QueryExecutionContext = {"Database": SCHEMA_NAME},
    ResultConfiguration = {
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    }
)

covid_db_testing_data_us_total_latest = load_query_result(athena_client, response)

In [ ]:
response = athena_client.start_query_execution(
    QueryString = 'SELECT * FROM usa_hospital_bedsusa_hospital_beds_csv',
    QueryExecutionContext = {"Database": SCHEMA_NAME},
    ResultConfiguration = {
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    }
)

usa_hospital_bedsusa_hospital_beds_csv = load_query_result(athena_client, response)

In [ ]:
response = athena_client.start_query_execution(
    QueryString = 'SELECT * FROM testing_data_us_dailyus_daily',
    QueryExecutionContext = {"Database": SCHEMA_NAME},
    ResultConfiguration = {
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    }
)

covid_db_testing_data_usus_daily = load_query_result(athena_client, response)

In [ ]:
response = athena_client.start_query_execution(
    QueryString = 'SELECT * FROM usa_hospital_bedsusa_hospital_beds_csv',
    QueryExecutionContext = {"Database": SCHEMA_NAME},
    ResultConfiguration = {
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    }
)

usa_hospital_bedsusa_hospital_beds_csv = load_query_result(athena_client, response)

In [ ]:
covid_db_testing_data_states_dailystates_daily.columns

Index(['date', 'state', 'positive', 'probablecases', 'negative', 'pending',
       'totaltestresultssource', 'totaltestresults', 'hospitalizedcurrently',
       'hospitalizedcumulative', 'inicucurrently', 'inicucumulative',
       'onventilatorcurrently', 'onventilatorcumulative', 'recovered',
       'lastupdateet', 'datemodified', 'checktimeet', 'death', 'hospitalized',
       'hospitalizeddischarged', 'datechecked', 'totaltestsviral',
       'positivetestsviral', 'negativetestsviral', 'positivecasesviral',
       'deathconfirmed', 'deathprobable', 'totaltestencountersviral',
       'totaltestspeopleviral', 'totaltestsantibody', 'positivetestsantibody',
       'negativetestsantibody', 'totaltestspeopleantibody',
       'positivetestspeopleantibody', 'negativetestspeopleantibody',
       'totaltestspeopleantigen', 'positivetestspeopleantigen',
       'totaltestsantigen', 'positivetestsantigen', 'fips', 'positiveincrease',
       'negativeincrease', 'total', 'totaltestresultsincrease', 

In [ ]:
enigma_jhud.columns

Index(['fips', 'admin2', 'province_state', 'country_region', 'last_update',
       'latitude', 'longitude', 'confirmed', 'deaths', 'recovered', 'active',
       'combined_key', 'partition_0'],
      dtype='object')

In [ ]:
fact_test = covid_db_testing_data_states_dailystates_daily[['fips', 'date', 'positive', 'negative', 
                                                            'hospitalizedcurrently', 'hospitalized',
                                                            'hospitalizeddischarged']]
fact_engima = enigma_jhud[['fips', 'province_state', 'country_region', 'confirmed', 'deaths', 'recovered', 'active']]

fact_covid = pd.merge(fact_test, fact_engima, on='fips', how='inner')

In [ ]:
fact_covid.head()

,fips,date,positive,negative,hospitalizedcurrently,hospitalized,hospitalizeddischarged,province_state,country_region,confirmed,deaths,recovered,active
0,72.0,20210307,101327,305972.0,147.0,NaN,NaN,Puerto Rico,US,3.0,0.0,0.0,NaN
1,72.0,20210307,101327,305972.0,147.0,NaN,NaN,Puerto Rico,US,5.0,0.0,0.0,NaN
2,72.0,20210307,101327,305972.0,147.0,NaN,NaN,Puerto Rico,US,5.0,0.0,0.0,NaN
3,72.0,20210307,101327,305972.0,147.0,NaN,NaN,Puerto Rico,US,14.0,0.0,0.0,NaN
4,72.0,20210307,101327,305972.0,147.0,NaN,NaN,Puerto Rico,US,21.0,1.0,0.0,NaN


In [ ]:
nytimes_data_in_usa_countryus_county.columns

Index(['date', 'county', 'state', 'fips', 'cases', 'deaths'], dtype='object')

In [ ]:
reg1 = enigma_jhud[['fips', 'province_state', 'country_region', 'latitude', 'longitude']]
reg2 = nytimes_data_in_usa_countryus_county[['fips', 'county', 'state']]
dim_reg_covid = pd.merge(reg1, reg2, on='fips', how='inner')

In [ ]:
dim_reg_covid.head()

,fips,province_state,country_region,latitude,longitude,county,state
0,NaN,Anhui,China,31.826,117.226,New York City,New York
1,NaN,Anhui,China,31.826,117.226,Unknown,Rhode Island
2,NaN,Anhui,China,31.826,117.226,New York City,New York
3,NaN,Anhui,China,31.826,117.226,Unknown,Rhode Island
4,NaN,Anhui,China,31.826,117.226,New York City,New York


In [ ]:
dim_reg_covid.shape

(3548832, 7)

In [ ]:
dim_date = covid_db_testing_data_states_dailystates_daily[['fips', 'date']]

In [ ]:
dim_date.head()

,fips,date
0,2.0,20210307
1,1.0,20210307
2,5.0,20210307
3,60.0,20210307
4,4.0,20210307


In [ ]:
dim_date['date'] = pd.to_datetime(dim_date['date'], format = '%Y%m%d')

C:\Users\segun\AppData\Local\Temp\ipykernel_3024\2345376272.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dim_date['date'] = pd.to_datetime(dim_date['date'], format = '%Y%m%d')


In [ ]:
dim_date.head()

,fips,date
0,2.0,2021-03-07
1,1.0,2021-03-07
2,5.0,2021-03-07
3,60.0,2021-03-07
4,4.0,2021-03-07


In [ ]:
dim_date['year'] = dim_date['date'].dt.year
dim_date['month'] = dim_date['date'].dt.month
dim_date['day_of_week'] = dim_date['date'].dt.dayofweek

C:\Users\segun\AppData\Local\Temp\ipykernel_3024\658808806.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dim_date['year'] = dim_date['date'].dt.year
C:\Users\segun\AppData\Local\Temp\ipykernel_3024\658808806.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dim_date['month'] = dim_date['date'].dt.month
C:\Users\segun\AppData\Local\Temp\ipykernel_3024\658808806.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = v

In [ ]:
dim_date.head()

,fips,date,year,month,day_of_week
0,2.0,2021-03-07,2021,3,6
1,1.0,2021-03-07,2021,3,6
2,5.0,2021-03-07,2021,3,6
3,60.0,2021-03-07,2021,3,6
4,4.0,2021-03-07,2021,3,6


In [ ]:
dim_hosp = usa_hospital_bedsusa_hospital_beds_csv[['fips', 'hospital_name', 'hospital_type', 'hq_address', 'hq_city', 'hq_state']]

In [ ]:
bucket = "covid19-de-pj"

In [ ]:
csv_buffer = StringIO()
fact_covid.to_csv(csv_buffer)

In [ ]:
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'output/fact_covid.csv').put(Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': 'SFZ3GWPH1KXATMK6',
  'HostId': 'wEHnBpvdBB1sjCwvrnzCQcA1QVkbDf7SR2CW+hHT6M34mDyo1JGnPee1EQv1imNA/75OQOC0Fu0=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'wEHnBpvdBB1sjCwvrnzCQcA1QVkbDf7SR2CW+hHT6M34mDyo1JGnPee1EQv1imNA/75OQOC0Fu0=',
   'x-amz-request-id': 'SFZ3GWPH1KXATMK6',
   'date': 'Tue, 21 Mar 2023 16:50:07 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"1d3733333aca8b37645d5c74d05145c6"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 1},
 'ETag': '"1d3733333aca8b37645d5c74d05145c6"',
 'ServerSideEncryption': 'AES256'}

In [ ]:
csv_buff = StringIO()
dim_date.to_csv(csv_buff)
#s3_resource.Object(bucket, 'output/dim_hosp.csv').put(Body=csv_buff.getvalue())
s3_resource.Object(bucket, 'output/dim_date.csv').put(Body=csv_buff.getvalue())
#s3_resource.Object(bucket, 'output/dim_reg_covid.csv').put(Body=csv_buffer.getvalue())
#s3_resource.Object(bucket, 'output/fact_covid.csv').put(Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': '9X459KXCA835XVW8',
  'HostId': '/hh1kdObNoZsTzHdbnMBN8Z1i1wMOkBF+fijgj8GkyqSgRadHYYvf2pZ4rCSjj5j0ACEQ+g7oNQ=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '/hh1kdObNoZsTzHdbnMBN8Z1i1wMOkBF+fijgj8GkyqSgRadHYYvf2pZ4rCSjj5j0ACEQ+g7oNQ=',
   'x-amz-request-id': '9X459KXCA835XVW8',
   'date': 'Tue, 21 Mar 2023 17:37:30 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"19eb0b77e7f7441c686829bc3fd1a906"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"19eb0b77e7f7441c686829bc3fd1a906"',
 'ServerSideEncryption': 'AES256'}

In [ ]:
csv_buffe = StringIO()
dim_reg_covid.to_csv(csv_buffe)
s3_resource.Object(bucket, 'output/dim_reg_covid.csv').put(Body=csv_buffe.getvalue())

{'ResponseMetadata': {'RequestId': 'SE389X7R2G6MSK71',
  'HostId': '/N3dxCriRmjJgeKiFuB+ESwB2vACkLeiPcwYrmu6v0EN9amzEeNfMLkZlh4BzZpVrKLkjnXz3Fs=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '/N3dxCriRmjJgeKiFuB+ESwB2vACkLeiPcwYrmu6v0EN9amzEeNfMLkZlh4BzZpVrKLkjnXz3Fs=',
   'x-amz-request-id': 'SE389X7R2G6MSK71',
   'date': 'Tue, 21 Mar 2023 17:40:21 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"0d7d4736a8179953c8c3f9a0d534c9d7"',
   'server': 'AmazonS3',
   'content-length': '0',
   'connection': 'close'},
  'RetryAttempts': 0},
 'ETag': '"0d7d4736a8179953c8c3f9a0d534c9d7"',
 'ServerSideEncryption': 'AES256'}

In [ ]:
dim_datesql = pd.io.sql.get_schema(dim_date.reset_index(), 'dim_date')
print(''.join(dim_datesql))

CREATE TABLE "dim_date" (
"index" INTEGER,
  "fips" REAL,
  "date" TIMESTAMP,
  "year" INTEGER,
  "month" INTEGER,
  "day_of_week" INTEGER
)


In [ ]:
fact_covidsql = pd.io.sql.get_schema(fact_covid.reset_index(), 'fact_covid')
print(''.join(fact_covidsql))

CREATE TABLE "fact_covid" (
"index" INTEGER,
  "fips" REAL,
  "date" INTEGER,
  "positive" INTEGER,
  "negative" REAL,
  "hospitalizedcurrently" REAL,
  "hospitalized" REAL,
  "hospitalizeddischarged" REAL,
  "province_state" TEXT,
  "country_region" TEXT,
  "confirmed" REAL,
  "deaths" REAL,
  "recovered" REAL,
  "active" REAL
)


In [ ]:
dim_reg_covidsql = pd.io.sql.get_schema(dim_reg_covid.reset_index(), 'dim_reg_covid')
print(''.join(dim_reg_covidsql))

CREATE TABLE "dim_reg_covid" (
"index" INTEGER,
  "fips" REAL,
  "province_state" TEXT,
  "country_region" TEXT,
  "latitude" REAL,
  "longitude" REAL,
  "county" TEXT,
  "state" TEXT
)


In [ ]:
dim_hospsql = pd.io.sql.get_schema(dim_hosp.reset_index(), 'dim_hosp')
print(''.join(dim_hospsql))

CREATE TABLE "dim_hosp" (
"index" TEXT,
  "fips" TEXT,
  "hospital_name" TEXT,
  "hospital_type" TEXT,
  "hq_address" TEXT,
  "hq_city" TEXT,
  "hq_state" TEXT
)


In [ ]:
import redshift_connector

In [ ]:
conn = redshift_connector.connect(
        host = '',
        database = 'dev',
        port = 5439,
        user = 'awsuser',
        password = ''
        )